In [ ]:
# !pip install -U "git+https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

!git clone -b alex "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"
!cp -r /content/csgy6953_DeepLearning_Midterm/src/ .
# !cp csgy6953_DeepLearning_Midterm/requirements.txt .

In [ ]:
# 
# !pip install -r requirements.txt
!pip install wandb
!wandb login "996181dd165ce17c309c3d027297e4ed8952f4ec"

In [1]:
import torch
import torch.nn as nn

import time
import random


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from src.data import get_transformed_data, make_data_loaders
from src.transforms import make_transforms, make_auto_transforms

batch_size = 512

valid_ratio = 0.1
train_data, valid_data, test_data = (
    get_transformed_data(
        make_transforms = make_auto_transforms, 
        valid_ratio = valid_ratio
    )
)

train_iterator, valid_iterator, test_iterator = (
    make_data_loaders(
        train_data,
        valid_data,
        test_data,
        batch_size
    )
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


**Define our Model

In [32]:
from src.model import ResNet, StemConfig, ResidualBlockType
from src.utils import initialize_parameters, epoch_time

architecture = [
    (ResidualBlockType.BOTTLENECK, 2, 128, 0.25),
    (ResidualBlockType.BOTTLENECK, 6, 256, 0.25),
    (ResidualBlockType.BOTTLENECK, 6, 512, 0.25),
    (ResidualBlockType.BOTTLENECK, 2, 1024, 0.25),
]

config = StemConfig(num_channels=128, kernel_size=3, stride=1, padding=1)
model = ResNet(architecture, stem_config=config, output_size=10, use_bias=True)

In [33]:
from pathlib import Path

path = "/content/drive/MyDrive/School/Tandon MSCS/Classes/CS-GY 6953: Deep Learning/midterm/"
# path = "/content/drive/MyDrive/Colab Notebooks/midterm/"
project_name = "alex_b512_drop__autoaug__bias__bottleb__adamW_cyclelr_arch1"
file_path = path + project_name

model_file = Path(file_path)

Need to run a dummy set of data to initialize the lazy modules before we can use torchsummary

In [34]:
if model_file.exists() and model_file.is_file():
  # load our previously trained model
  model.load_state_dict(torch.load(model_file))
  model = model.to(device)
else:
  # intialize a new model
  inputs = torch.empty((batch_size, 3, 32, 32))
  inputs.normal_()
  model = model.to(device)
  y = model(inputs.to(device))
  print(y.size())

  model.apply(initialize_parameters)

torch.Size([512, 10])


In [35]:
from src.utils import count_parameters

num_params, grad_params = count_parameters(model)
print(f"There are {grad_params:,} trainable parameters.")

There are 4,915,082 trainable parameters.


In [ ]:
from src.engine import train_one_epoch, evaluate
from torch.optim.lr_scheduler import CyclicLR

best_loss = float('inf')
EPOCHS  = 100
learning_rate = 0.01
max_lr = 0.5
base_lr = 0.0001
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=learning_rate, cycle_momentum=False)

if model_file.is_file():
  # if we loaded a previously saved iteration, we want to get the current
  # best loss otherwise we could overwrite our save with a worse model
  loss, acc = evaluate(model.to(device), test_iterator, criterion, device)
  best_loss = loss
  print(f"Prevous best loss: {loss:.4f}, acc: {acc * 100:.2f}%")

In [ ]:
# setup wandb logging
import wandb

wandb.init(
    project='ResNet_5M',
    name=project_name,
    entity="dlf22_mini_project",
    config={
        "learning_rate":learning_rate,
        "epochs": EPOCHS,
        "batch_size": batch_size,
        "architecture": architecture,
        "grad_params": grad_params,
        "lr_schedule": f"CyclicLR_base_{base_lr}_max_{max_lr}"
        }
)

In [ ]:
for epoch in range(1, EPOCHS+1):
    start = time.time()

    print(f"Epoch {epoch}")
    train_loss, train_acc = train_one_epoch(model, train_iterator, criterion, optimizer, device)
    train_mins, train_secs = epoch_time(start, time.time())

    wandb.log({
        "train_loss": train_loss,
        "train_acc": train_acc,
        "epoch": epoch
    })

    print(f"\tTrain elapsed: {train_mins}:{train_secs}, loss: {train_loss:.4f}, acc: {train_acc * 100:.2f}%")

    start = time.time()
    val_loss, val_acc = evaluate(model, valid_iterator, criterion, device)
    val_mins, val_secs = epoch_time(start, time.time())

    wandb.log({
        "val_loss": val_loss,
        "val_acc": val_acc,
        "epoch": epoch,
    })
    
    scheduler.step()
    print(f"\tValidation elapsed: {val_mins}:{val_secs}, loss: {val_loss:.4f}, acc: {val_acc * 100:.2f}%")

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), file_path)

## Evaluate the Model  

In [ ]:
model.load_state_dict(torch.load(file_path))
test_loss, test_acc = evaluate(model.to(device), test_iterator, criterion, device)
print(f"Test Loss: {test_loss:.4f}\nTest Accuracy: {test_acc * 100:.2f}%")

wandb.log({
    "test_loss": test_loss,
    "test_acc": test_acc,
})